In [34]:
import re
txt = './beeline_consent_query_stderr.txt'
QES = {}
TES = {}
DM = {}

In [35]:
lines = []
with open(txt,'r') as txt:
    for line in txt:
        lines.append(line)

##### First part of the task: Query Execution Summary ( QES ) dictionary: 

In [29]:
QES_Keys = ['Compile Query','Prepare Plan','Get Query Coordinator (AM)','Submit Plan','Start DAG','Run DAG']
for line in lines:
    for qes_key in QES_Keys:
        if qes_key in line:
            QES[qes_key] = line.strip().split()[-1]
#searching for the key-values pairs from the beginning 

##### Second part of the task: Task Execution Summary ( TES ) complex dictionary:

In [36]:
i = 0
seps = []
for (num , line) in enumerate(lines):
    if 'Task Execution Summary' in line:
        TES_line_number = num #getting TES header location
    if 'Completed executing command' in line:
        com_line_number = num #this is for the 3rd part of the description, figures out the last line of the DMs
for line in lines[TES_line_number:]:
    if '---------' in line:
        seps.append(i) #getting the location of the lines/separators after TES' line location
    i+=1
for line in lines[(TES_line_number+1+seps[-2]):(TES_line_number+seps[-1])]:
    line_array = line.strip().split()
    TES[line_array[2]+" "+line_array[3]]={"DURATION(ms)":line_array[4],"CPU_TIME(ms)":line_array[5],
                                          "GC_TIME(ms)":line_array[6],"INPUT_RECORDS":line_array[7],
                                          "OUTPUT_RECORDS":line_array[8]}

##### Third part of the task: Detailed Metrics per task ( DM ) complex dictionary:

In [38]:
last_sep_line_num = TES_line_number+2+seps[-1] #getting the starting position of our data which is located 2 rows after the last separator/line.
#used that as a starting point because I'm confident that that's what a normal layout of this file looks like and I wasn't sure that the Detailed Metrics are all present all the time.
for line in lines[last_sep_line_num:(com_line_number)]:
    if re.search(r"^INFO\s+:\s\w",line):
        key = line.split(':')[1].strip()
        DM[key] = {}
    elif re.search(r"INFO\s+:\s{3,5}\w", line): #it would accept 3 to 5 spaces here 
        k, v = line.split(':')[1:]
        DM[key][k.strip()] = int(v)
    else:
        print('different format')